In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
from unidecode import unidecode

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
with open('json0929/permanentes.json') as f:
    data = json.load(f)

In [11]:
def filter_list_of_dicts(list_of_dicts, property_name, property_value):
    """
    This function filters a list of dictionaries by a given property name and value using the filter() function.
    :param list_of_dicts: list of dictionaries to be filtered
    :param property_name: name of the property to filter by
    :param property_value: value of the property to filter by
    :return: a list of dictionaries that match the filter criteria
    """
    return filter(lambda d: d.get(property_name) == property_value, list_of_dicts)


In [22]:
def get_item_by_key(key: int) -> dict:
    """
    This function returns a dictionary item from a list of dictionaries based on a given key.
    :param key: key to filter by
    :return: a dictionary item that matches the key
    """
    return next(filter_list_of_dicts(data, 'key', key), None)

In [30]:
cadeira = get_item_by_key(337921)
cadeira['detalhes']

{'patrimonio': 543858,
 'n_controle': None,
 'cod_barras': 337921,
 'n_serie': None,
 'descricao': 'CADEIRA ESCOLAR UNIVERSITÁRIA - CADEIRA ESCOLAR COM PRANCHETA FIXA EM MDF, ASSENTO E ENCOSTO EM COMPENSADO E ESPUMA, REVESTIDO NA COR AZUL ROYAL. ESTRUTURA CONFECCIONADA EM TUBO DE AÇO PINTADO EM PRETO.',
 'conservacao': 'EM USO',
 'incorporacao': '12/02/2018',
 'transf_local': None,
 'valor': 140,
 'situacao': 'Ativo',
 'peso': '',
 'medidas': ''}

# Cadeiras

## Escolar

In [32]:
def batch_write_update_medidas(collection_name: str, id_list: list, medidas: str):
    batch = db.batch() 
    collection_ref = db.collection(collection_name)
    k = 1
    for i in range(0, len(id_list), 500):
        chunk = id_list[i:i+500]
        for idx in chunk: 
            doc_ref = collection_ref.document(str(idx))
            batch.update(doc_ref, {'detalhes.medidas': medidas})
        print(f"{collection_name}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [36]:
cadeiras_escolar_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', "CADEIRA ESCOLAR UNIVERSITÁRIA") ]
    

In [37]:
batch_write_update_medidas("items", cadeiras_escolar_ids, '95 x 80 x 56 (AxCxL)')

items: chunk 1 (500 documents)
items: chunk 2 (474 documents)


## Cadeira com braços

In [39]:
get_item_by_key(310264)["short_descricao"]
# 'CADEIRA ESCRITÓRIO FIXA COM BRAÇO'

'CADEIRA ESCRITÓRIO FIXA COM BRAÇO'

In [42]:
cadeiras_com_braço_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(310264)["short_descricao"])]


In [46]:
batch_write_update_medidas("items", cadeiras_com_braço_ids, '96 x 59 x 63 (AxCxL)')

items: chunk 1 (98 documents)


## Cadeira sem braços

In [47]:
get_item_by_key(347857)["short_descricao"]


'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS'

In [48]:
cadeiras_sem_braço_ids = [ x['key'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(347857)["short_descricao"])]


In [50]:
len(cadeiras_sem_braço_ids)

232

In [51]:
set([ x['detalhes']['descricao'] for x in  filter_list_of_dicts(data, 'short_descricao', get_item_by_key(347857)["short_descricao"])])

{'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM,TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM.',
 'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM,TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM. ',
 'CADEIRA DE DIÁLOGO FIXA SEM BRAÇOS - MARCA: FLEXFORM; MODELO: DOTY/CL.DOTY.C.EF. ESTOFADA, EMPILHÁVEL, SEM BRAÇOS, MONTADA SOBRE ARMAÇÃO TUBULAR DE AÇO, DE COR PRETA, ASSENTO 460MM X 460 MM, ENCOSTO 400MM, TECIDO DE CREPE 100% POLIESTER, ESTRUTURA CONFECCIONADA EM AÇO CARBONO, TINTURA COM TRATAMENTO ANTIFERRUGEM.'}

In [52]:
batch_write_update_medidas("items", cadeiras_sem_braço_ids, '79 x 55 x 51 (AxCxL)')

items: chunk 1 (232 documents)


## Mesa reta 4 pés

In [53]:
get_item_by_key(363625)["short_descricao"]

'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM '

In [67]:
mesa_reta_4 = filter_list_of_dicts(data, 'short_descricao', get_item_by_key(363625)["short_descricao"])

In [60]:
set([ x['detalhes']['descricao'] for x in  mesa_reta_4])

{'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MARCA: TECNO2000, MODELO: BREMEN. MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CLARO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MARCA: TECNO2000; MODELO: BREMEN. MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CLARO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR CINZA CRISTAL, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.',
 'MESA RETA 4 PÉS, 800(L)X600(P)X740(A)MM  - MESA RETANGULAR, EM MADEIRA MDP REVESTIDA NA COR OVO, ESTRUTURA EM AÇO COM PINTURA EPÓXI-PÓ NA COR ALUMÍNIO NATURAL. DIMENSÕES: 80X 60 X 74 CM.'}

In [68]:
mesa_reta_4_ids = [ x['key'] for x in  mesa_reta_4]

In [70]:
batch_write_update_medidas("items", mesa_reta_4_ids, '75 x 60 x 80 (AxCxL)')

items: chunk 1 (88 documents)
